# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [35]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [36]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,urangan,-25.3000,152.9000,24.03,85,100,6.87,AU,1734203602
1,1,thompson,55.7435,-97.8558,-11.91,85,100,0.00,CA,1734203603
2,2,arta,39.1606,20.9853,13.27,94,100,2.42,GR,1734203320
3,3,mitu,1.1983,-70.1733,32.02,48,17,1.57,CO,1734203512
4,4,lihue,21.9789,-159.3672,24.77,78,75,7.72,US,1734203606


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [37]:


# Configure the map plot
map_plot_3 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_3

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [38]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = city_data_df[
    (city_data_df["Max Temp"] > 21) & 
    (city_data_df["Max Temp"] < 27) & 
    (city_data_df["Wind Speed"] < 4.5) & 
    (city_data_df["Cloudiness"] == 0)
]


# Drop any rows with null values
ideal_conditions = ideal_conditions.dropna()

# Display sample data
ideal_conditions.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
80,80,salalah,17.0151,54.0924,24.05,53,0,2.06,OM,1734203691
191,191,cuencame de ceniceros,24.8711,-103.6964,23.23,26,0,1.83,MX,1734203810
265,265,charters towers,-20.1000,146.2667,23.96,86,0,2.26,AU,1734203895
267,267,springbok,-29.6643,17.8865,21.65,27,0,2.13,ZA,1734203897
276,276,el bolson,-41.9667,-71.5167,26.12,24,0,2.00,AR,1734203906


### Step 3: Create a new DataFrame called `hotel_df`.

In [39]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_conditions[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
80,salalah,OM,17.0151,54.0924,53,
191,cuencame de ceniceros,MX,24.8711,-103.6964,26,
265,charters towers,AU,-20.1000,146.2667,86,
267,springbok,ZA,-29.6643,17.8865,27,
276,el bolson,AR,-41.9667,-71.5167,24,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:

# Set the geographical coordinates
latitude = ["Lat"]
longitude = ["Lng"]

radius = 10000

# Set parameters to search for a hotel
params = {
        "categories": "accommodation.hotel",
        "filter": f"circle:{longitude},{latitude},{radius}",  # Circular search area
        "bias": f"proximity:{longitude},{latitude}",          # Bias results closer to the location
        "limit": 1,                                           # Limit results to 1 hotel
        "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
salalah - nearest hotel: Muscat International Hotel
cuencame de ceniceros - nearest hotel: HOTEL La Casa Viajero
charters towers - nearest hotel: No hotel found
springbok - nearest hotel: Elkoweru Guesthouse
el bolson - nearest hotel: Cabañas Willy


,City,Country,Lat,Lng,Humidity,Hotel Name
80,salalah,OM,17.0151,54.0924,53,Muscat International Hotel
191,cuencame de ceniceros,MX,24.8711,-103.6964,26,HOTEL La Casa Viajero
265,charters towers,AU,-20.1000,146.2667,86,No hotel found
267,springbok,ZA,-29.6643,17.8865,27,Elkoweru Guesthouse
276,el bolson,AR,-41.9667,-71.5167,24,Cabañas Willy


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:


# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",          
    "Lat",          
    geo=True,       
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",     
    color="City",
    tiles="OSM",
    hover_cols=["City", "Country", "Hotel Name", "Humidity"] 
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)